<a href="https://colab.research.google.com/github/rheav24/-Project-Repo/blob/main/INST326_Exercises_W05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INST326 — Week 5 Exercises: Encapsulation & Data Hiding (Library Management Project)

**Scope guard:** These exercises stay within *Week 5 topics only* — encapsulation, private/protected attributes, getters/setters, and `@property` with validation. **Avoid** features introduced in later weeks (e.g., class/staticmethods, inheritance, abstract classes, advanced error handling/test frameworks).


## Python skills you’ll need (Week 5 scope)

- Defining classes and creating instances (`class`, `__init__`, `self`)
- Naming conventions for attribute privacy (`_protected`, `__private`/name mangling)
- Encapsulation patterns:
  - Getter/setter methods
  - `@property` / `@<name>.setter`
  - Simple validation logic inside setters (e.g., type/empty checks, ranges)
- Basic built-in types and operations (`str`, `int`, `float`, `bool`, `len`, `in`)
- F-strings for messages (optional)
- Simple list/dict usage for holding objects (no advanced collections needed)
- Basic control flow (`if`, `for`)
- (Optional but allowed) raising simple exceptions like `ValueError` in setters for invalid data (no try/except blocks required here)


---
## How to use this notebook

Each exercise includes a short prompt and a starter code cell. Keep your solutions within **Week 5** boundaries. If an exercise references existing domain classes, you may stub minimal versions as needed inside the same cell (or reuse earlier cells). Keep code readable and well-encapsulated.


### Exercise 1 — Encapsulated `Book` identifiers
Create a `Book` class with *private* attributes `__isbn` (string) and `__title` (string). Expose read-only access via `@property` for both. Prevent empty strings at construction time using validation inside `__init__`. If invalid, either normalize to a sensible default or raise `ValueError` (your choice).

In [1]:
class Book:
    def __init__(self, isbn: str, title: str):
        """
        Initialize a Book with validation.

        Args:
            isbn: The ISBN string (must be non-empty)
            title: The book title (must be non-empty)

        Raises:
            ValueError: If isbn or title is an empty string
        """

        if not isinstance(isbn, str) or isbn.strip() == "":
            raise ValueError("ISBN must be a non-empty string")
        self.__isbn = isbn.strip()


        if not isinstance(title, str) or title.strip() == "":
            raise ValueError("Title must be a non-empty string")
        self.__title = title.strip()

    @property
    def isbn(self) -> str:
        """Read-only access to the book's ISBN."""
        return self.__isbn

    @property
    def title(self) -> str:
        """Read-only access to the book's title."""
        return self.__title



if __name__ == "__main__":
    b = Book("9780132350884", "Clean Code")
    print(f"{b.isbn}, {b.title}")


9780132350884, Clean Code


### Exercise 2 — Writable `Member` email with validation
Define a `Member` class with private `__email`. Provide `email` property and setter that requires `@` in the email and trims whitespace. Reject obviously invalid emails by raising `ValueError`. Provide a `__repr__` that shows member id and email.

In [4]:
class Member:
    def __init__(self, member_id: int, email: str):

        self.member_id = member_id
        self.email = email

    def __repr__(self) -> str:
        """Developer-facing representation showing member id and email."""
        return f"Member(id={self.member_id}, email={self.__email!r})"

    @property
    def email(self) -> str:
        """Read the member's email address."""
        return self.__email

    @email.setter
    def email(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError("Email must be a string")

        trimmed = value.strip()


        if not trimmed:
            raise ValueError("Email cannot be empty or whitespace-only")

        if "@" not in trimmed:
            raise ValueError("Email must contain '@' character")

        if trimmed.startswith("@") or trimmed.endswith("@"):
            raise ValueError("Email format is invalid: '@' cannot be at start or end")

        parts = trimmed.split("@")
        if len(parts) != 2 or not parts[0] or not parts[1]:
            raise ValueError("Email format is invalid: missing local or domain part")

        self.__email = trimmed


if __name__ == "__main__":
    m = Member(101, "  user@example.org ")
    print(m)
    print(m.email)

    print()

    m.email = "  newuser@domain.com  "
    print(m)
    print(m.email)

    print()

Member(id=101, email='user@example.org')
user@example.org

Member(id=101, email='newuser@domain.com')
newuser@domain.com



### Exercise 3 — Protected page count with non-negative constraint
Extend `Book` (or create a fresh class here) with a *protected* attribute `_pages` and a `pages` property. Setter must coerce to `int` and ensure it’s ≥ 1 (treat 0/negatives as invalid).

In [6]:
class PagedBook:
    def __init__(self, title: str, pages):

        self.title = title
        self.pages = pages  # Uses the setter for validation

    @property
    def pages(self) -> int:
        """Get the number of pages."""
        return self._pages

    @pages.setter
    def pages(self, value) -> None:

        try:
            pages_int = int(value)
        except (ValueError, TypeError):
            raise ValueError(f"Pages must be coercible to int, got {type(value).__name__}")

        if pages_int < 1:
            raise ValueError(f"Pages must be >= 1, got {pages_int}")

        self._pages = pages_int

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"PagedBook(title={self.title!r}, pages={self._pages})"



if __name__ == "__main__":

    pb = PagedBook("Fluent Python", 792)
    print(pb.pages)
    print(pb)

    print()


    pb2 = PagedBook("Clean Code", "342")
    print(pb2.pages)
    print(pb2)

    print()


    pb3 = PagedBook("Design Patterns", 395.7)
    print(pb3.pages)
    print(pb3)

    print()


    pb.pages = "850"
    print(pb.pages)

    print()


792
PagedBook(title='Fluent Python', pages=792)

342
PagedBook(title='Clean Code', pages=342)

395
PagedBook(title='Design Patterns', pages=395)

850



### Exercise 4 — Loan period property with range check
Create a `LoanPolicy` class with private `__max_days`. Provide property/settter ensuring 1 ≤ `max_days` ≤ 60. Default to 21 days.

In [7]:
class LoanPolicy:
    def __init__(self, max_days: int = 21):

        self.max_days = max_days

    @property
    def max_days(self) -> int:
        """Get the maximum loan duration in days."""
        return self.__max_days

    @max_days.setter
    def max_days(self, value: int) -> None:

        if not isinstance(value, int):
            raise ValueError(f"max_days must be an integer, got {type(value).__name__}")

        if value < 1 or value > 60:
            raise ValueError(f"max_days must be between 1 and 60 (inclusive), got {value}")

        self.__max_days = value

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"LoanPolicy(max_days={self.__max_days})"



if __name__ == "__main__":

    p = LoanPolicy()
    print(p.max_days)
    print(p)

    print()


    p2 = LoanPolicy(14)
    print(p2.max_days)
    print(p2)

    print()


    p.max_days = 30
    print(p.max_days)
    print(p)

    print()


    p_min = LoanPolicy(1)
    print(f"Minimum: {p_min.max_days}")

    p_max = LoanPolicy(60)
    print(f"Maximum: {p_max.max_days}")

    print()


21
LoanPolicy(max_days=21)

14
LoanPolicy(max_days=14)

30
LoanPolicy(max_days=30)

Minimum: 1
Maximum: 60



### Exercise 5 — Read-only `copy_id`
Create a `Copy` class representing a physical copy of a book with private `__copy_id` and protected `_condition` ('good', 'fair', 'poor'). Only `copy_id` is read-only. `condition` is a property with setter that restricts values to the allowed set.

In [8]:
class Copy:
    ALLOWED_CONDITIONS = {'good', 'fair', 'poor'}

    def __init__(self, copy_id: str, condition: str = "good"):

        self.__copy_id = copy_id
        self.condition = condition

    @property
    def copy_id(self) -> str:
        """Get the copy's unique identifier (read-only)."""
        return self.__copy_id

    @property
    def condition(self) -> str:
        """Get the physical condition of this copy."""
        return self._condition

    @condition.setter
    def condition(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"condition must be a string, got {type(value).__name__}")

        value_lower = value.lower().strip()

        if value_lower not in self.ALLOWED_CONDITIONS:
            raise ValueError(
                f"condition must be one of {sorted(self.ALLOWED_CONDITIONS)}, "
                f"got '{value}'"
            )

        self._condition = value_lower

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Copy(copy_id={self.__copy_id!r}, condition={self._condition!r})"

if __name__ == "__main__":
    c = Copy("C-1001")
    print(c.copy_id, c.condition)
    print(c)

    print()


    c2 = Copy("C-1002", "fair")
    print(c2.copy_id, c2.condition)
    print(c2)

    print()


    c.condition = "poor"
    print(c.condition)
    print(c)

    print()

    c3 = Copy("C-1003", "  GOOD  ")
    print(c3.condition)

    print()





C-1001 good
Copy(copy_id='C-1001', condition='good')

C-1002 fair
Copy(copy_id='C-1002', condition='fair')

poor
Copy(copy_id='C-1001', condition='poor')

good



### Exercise 6 — Late fee rate with type safety
Create a `FeeSchedule` with private `__late_fee_per_day` (float). Property/settter must coerce numeric strings like '0.25' to float, and reject negatives.

In [10]:

class FeeSchedule:
    def __init__(self, late_fee_per_day):

        self.late_fee_per_day = late_fee_per_day

    @property
    def late_fee_per_day(self) -> float:
        """Get the late fee per day."""
        return self.__late_fee_per_day

    @late_fee_per_day.setter
    def late_fee_per_day(self, value) -> None:

        try:
            fee_float = float(value)
        except (ValueError, TypeError):
            raise ValueError(
                f"late_fee_per_day must be coercible to float, "
                f"got {type(value).__name__}: {value!r}"
            )

        if fee_float < 0.0:
            raise ValueError(f"late_fee_per_day must be >= 0.0, got {fee_float}")

        self.__late_fee_per_day = fee_float

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"FeeSchedule(late_fee_per_day={self.__late_fee_per_day})"

if __name__ == "__main__":
    fs = FeeSchedule("0.35")
    print(fs.late_fee_per_day)
    print(fs)

    print()

    fs2 = FeeSchedule(1)
    print(fs2.late_fee_per_day)

    print()


    fs3 = FeeSchedule(0.50)
    print(fs3.late_fee_per_day)

    print()


    fs4 = FeeSchedule("0.0")
    print(fs4.late_fee_per_day)

    print()


    fs.late_fee_per_day = "0.75"
    print(fs.late_fee_per_day)
    print(fs)

    print()


    fs5 = FeeSchedule("  0.25  ")
    print(fs5.late_fee_per_day)

    print()


0.35
FeeSchedule(late_fee_per_day=0.35)

1.0

0.5

0.0

0.75
FeeSchedule(late_fee_per_day=0.75)

0.25



### Exercise 7 — Aggregating encapsulated objects (simple list)
Create a `Shelf` with a *protected* list `_copies`. Implement `add_copy(copy)` that accepts only `Copy` instances. Expose a read-only `copies` property returning a **tuple** (to prevent external mutation).

In [11]:
class Copy:
    """Represents a physical copy of a book."""
    ALLOWED_CONDITIONS = {'good', 'fair', 'poor'}

    def __init__(self, copy_id: str, condition: str = "good"):
        self.__copy_id = copy_id
        self.condition = condition

    @property
    def copy_id(self) -> str:
        return self.__copy_id

    @property
    def condition(self) -> str:
        return self._condition

    @condition.setter
    def condition(self, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError(f"condition must be a string, got {type(value).__name__}")

        value_lower = value.lower().strip()
        if value_lower not in self.ALLOWED_CONDITIONS:
            raise ValueError(
                f"condition must be one of {sorted(self.ALLOWED_CONDITIONS)}, got '{value}'"
            )
        self._condition = value_lower

    def __repr__(self) -> str:
        return f"Copy(copy_id={self.__copy_id!r}, condition={self._condition!r})"


class Shelf:
    def __init__(self):
        """Initialize an empty shelf."""
        self._copies = []

    def add_copy(self, copy) -> None:

        if not isinstance(copy, Copy):
            raise TypeError(
                f"copy must be a Copy instance, got {type(copy).__name__}"
            )
        self._copies.append(copy)

    @property
    def copies(self):

        return tuple(self._copies)

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Shelf(copies={len(self._copies)})"


if __name__ == "__main__":
    sh = Shelf()
    print(sh)
    print(sh.copies)

    print()

    sh.add_copy(Copy("C-1"))
    sh.add_copy(Copy("C-2", "fair"))
    sh.add_copy(Copy("C-3", "poor"))

    print(sh)
    print(sh.copies)

    print()

    first_copy = sh.copies[0]
    print(first_copy.copy_id, first_copy.condition)

    print()

    print("Shelf contents:")
    for copy in sh.copies:
        print(f"  {copy.copy_id}: {copy.condition}")

    print()

    print()


Shelf(copies=0)
()

Shelf(copies=3)
(Copy(copy_id='C-1', condition='good'), Copy(copy_id='C-2', condition='fair'), Copy(copy_id='C-3', condition='poor'))

C-1 good

Shelf contents:
  C-1: good
  C-2: fair
  C-3: poor




### Exercise 8 — Borrower name normalization
Create a `Borrower` with private `__first_name` and `__last_name`. Properties should normalize spacing and capitalization (`title()` is fine). Provide a read-only property `full_name`.

In [14]:
class Borrower:
    def __init__(self, first_name: str, last_name: str):

        self.first_name = first_name
        self.last_name = last_name

    @property
    def first_name(self) -> str:
        """Get the borrower's first name (normalized)."""
        return self.__first_name

    @first_name.setter
    def first_name(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"first_name must be a string, got {type(value).__name__}")

        normalized = value.strip().title()

        if not normalized:
            raise ValueError("first_name cannot be empty or whitespace-only")

        self.__first_name = normalized

    @property
    def last_name(self) -> str:
        """Get the borrower's last name (normalized)."""
        return self.__last_name

    @last_name.setter
    def last_name(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"last_name must be a string, got {type(value).__name__}")

        normalized = value.strip().title()

        if not normalized:
            raise ValueError("last_name cannot be empty or whitespace-only")

        self.__last_name = normalized

    @property
    def full_name(self) -> str:

        return f"{self.__first_name} {self.__last_name}"

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Borrower(first_name={self.__first_name!r}, last_name={self.__last_name!r})"

if __name__ == "__main__":
    b = Borrower("  Rhea ", "  Vyragaram ")
    print(b.full_name)
    print(b)
    print()

    print(b.first_name)
    print(b.last_name)

    print()


    b.first_name = "  Rhea  "
    print(b.full_name)
    print()


    b2 = Borrower("  MICHAEL  ", "o'brien")
    print(b2.full_name)

    print()


    b3 = Borrower("\t\n  charlie  \n", "   BROWN   ")
    print(b3.full_name)

    print()

    print()

Rhea Vyragaram
Borrower(first_name='Rhea', last_name='Vyragaram')

Rhea
Vyragaram

Rhea Vyragaram

Michael O'Brien

Charlie Brown




### Exercise 9 — Minimum age policy for membership
Create `MembershipPolicy` with private `__min_age` default 13. Property/settter must enforce 0 ≤ min_age ≤ 120 and coerce to `int`. Add method `is_eligible(age)` that checks age against the policy (no class/staticmethods).

In [15]:
class MembershipPolicy:
    def __init__(self, min_age: int = 13):

        self.min_age = min_age  # Uses the setter for validation

    @property
    def min_age(self) -> int:
        """Get the minimum age requirement for membership."""
        return self.__min_age

    @min_age.setter
    def min_age(self, value) -> None:

        try:
            age_int = int(value)
        except (ValueError, TypeError):
            raise ValueError(
                f"min_age must be coercible to int, got {type(value).__name__}: {value!r}"
            )

        if age_int < 0 or age_int > 120:
            raise ValueError(f"min_age must be between 0 and 120 (inclusive), got {age_int}")

        self.__min_age = age_int

    def is_eligible(self, age) -> bool:

        try:
            age_int = int(age)
        except (ValueError, TypeError):
            raise ValueError(
                f"age must be coercible to int, got {type(age).__name__}: {age!r}"
            )

        return age_int >= self.__min_age

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"MembershipPolicy(min_age={self.__min_age})"

if __name__ == "__main__":

    mp = MembershipPolicy()
    print(mp)
    print(mp.min_age)

    print()


    print(mp.is_eligible(12))
    print(mp.is_eligible(13))
    print(mp.is_eligible(15))

    print()


    print(mp.is_eligible("12"))
    print(mp.is_eligible("15"))

    print()


    mp.min_age = 18
    print(mp)
    print(mp.is_eligible(16))
    print(mp.is_eligible(18))

    print()


    mp2 = MembershipPolicy("21")
    print(mp2.min_age)

    print()


    mp3 = MembershipPolicy(13.9)
    print(mp3.min_age)

    print()


    mp_child = MembershipPolicy(0)
    print(f"No minimum: {mp_child.is_eligible(0)}")

    mp_senior = MembershipPolicy(120)
    print(f"Max valid: {mp_senior.min_age}")

    print()


MembershipPolicy(min_age=13)
13

False
True
True

False
True

MembershipPolicy(min_age=18)
False
True

21

13

No minimum: True
Max valid: 120



### Exercise 10 — Encapsulated `Loan` dates (strings allowed)
Create `Loan` with private `__start_date` and `__due_date` stored as ISO date strings 'YYYY-MM-DD'. Properties should validate format *lightly* via length and digit checks (no datetime imports).

In [16]:
class Loan:
    def __init__(self, start_date: str, due_date: str):

        self.start_date = start_date
        self.due_date = due_date

    @staticmethod
    def _validate_iso_date(value: str) -> str:

        if not isinstance(value, str):
            raise ValueError(f"Date must be a string, got {type(value).__name__}")


        if len(value) != 10:
            raise ValueError(
                f"Date must be 10 characters in 'YYYY-MM-DD' format, "
                f"got length {len(value)}: {value!r}"
            )


        if value[4] != '-' or value[7] != '-':
            raise ValueError(
                f"Date must be in 'YYYY-MM-DD' format, got {value!r}"
            )

        year_str = value[0:4]
        month_str = value[5:7]
        day_str = value[8:10]

        if not year_str.isdigit():
            raise ValueError(f"Year must be numeric in {value!r}")
        if not month_str.isdigit():
            raise ValueError(f"Month must be numeric in {value!r}")
        if not day_str.isdigit():
            raise ValueError(f"Day must be numeric in {value!r}")


        month = int(month_str)
        day = int(day_str)

        if month < 1 or month > 12:
            raise ValueError(f"Month must be between 01 and 12, got {month_str} in {value!r}")
        if day < 1 or day > 31:
            raise ValueError(f"Day must be between 01 and 31, got {day_str} in {value!r}")

        return value

    @property
    def start_date(self) -> str:
        """Get the loan start date in ISO format."""
        return self.__start_date

    @start_date.setter
    def start_date(self, value: str) -> None:

        self.__start_date = self._validate_iso_date(value)

    @property
    def due_date(self) -> str:
        """Get the loan due date in ISO format."""
        return self.__due_date

    @due_date.setter
    def due_date(self, value: str) -> None:

        self.__due_date = self._validate_iso_date(value)

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Loan(start_date={self.__start_date!r}, due_date={self.__due_date!r})"

if __name__ == "__main__":

    ln = Loan("2025-10-06", "2025-10-27")
    print(ln.start_date, ln.due_date)
    print(ln)

    print()


    ln.start_date = "2025-09-01"
    print(ln.start_date)

    print()


    ln2 = Loan("2024-01-01", "2024-12-31")
    print(ln2)

    ln3 = Loan("2025-02-28", "2025-03-15")
    print(ln3)

    print()

    ln_edge = Loan("1900-01-01", "2099-12-31")
    print(ln_edge)

    print()

2025-10-06 2025-10-27
Loan(start_date='2025-10-06', due_date='2025-10-27')

2025-09-01

Loan(start_date='2024-01-01', due_date='2024-12-31')
Loan(start_date='2025-02-28', due_date='2025-03-15')

Loan(start_date='1900-01-01', due_date='2099-12-31')



### Exercise 11 — Computed read-only property: `is_overdue`
On `Loan` (or your own class), add a read-only boolean property `is_overdue` that compares today-as-string provided to a method e.g., `mark_today(date_str)` that stores a private `__today`. No datetime libs; simple string comparison is acceptable.

In [17]:
class SimpleLoan:
    def __init__(self, due_date: str):

        self.__due_date = self._validate_iso_date(due_date)
        self.__today = None

    @staticmethod
    def _validate_iso_date(value: str) -> str:

        if not isinstance(value, str):
            raise ValueError(f"Date must be a string, got {type(value).__name__}")


        if len(value) != 10:
            raise ValueError(
                f"Date must be 10 characters in 'YYYY-MM-DD' format, "
                f"got length {len(value)}: {value!r}"
            )


        if value[4] != '-' or value[7] != '-':
            raise ValueError(
                f"Date must be in 'YYYY-MM-DD' format, got {value!r}"
            )

        year_str = value[0:4]
        month_str = value[5:7]
        day_str = value[8:10]

        if not year_str.isdigit():
            raise ValueError(f"Year must be numeric in {value!r}")
        if not month_str.isdigit():
            raise ValueError(f"Month must be numeric in {value!r}")
        if not day_str.isdigit():
            raise ValueError(f"Day must be numeric in {value!r}")


        month = int(month_str)
        day = int(day_str)

        if month < 1 or month > 12:
            raise ValueError(f"Month must be between 01 and 12, got {month_str} in {value!r}")
        if day < 1 or day > 31:
            raise ValueError(f"Day must be between 01 and 31, got {day_str} in {value!r}")

        return value

    def mark_today(self, date_str: str) -> None:

        self.__today = self._validate_iso_date(date_str)

    @property
    def is_overdue(self) -> bool:

        if self.__today is None:
            return False

        return self.__today > self.__due_date

    @property
    def due_date(self) -> str:
        """Get the loan's due date."""
        return self.__due_date

    @property
    def today(self) -> str:
        """Get the current date as marked (or None if not set)."""
        return self.__today

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return (
            f"SimpleLoan(due_date={self.__due_date!r}, "
            f"today={self.__today!r}, is_overdue={self.is_overdue})"
        )


if __name__ == "__main__":
    sl = SimpleLoan("2025-10-20")
    print(f"Due date: {sl.due_date}")
    print(f"Today not set yet, is_overdue: {sl.is_overdue}")
    print()


    sl.mark_today("2025-10-21")
    print(f"Today: {sl.today}")
    print(f"Is overdue: {sl.is_overdue}")
    print(sl)
    print()

    sl2 = SimpleLoan("2025-10-27")
    print(f"Due date: {sl2.due_date}")
    print(f"Is overdue (before marking today): {sl2.is_overdue}")
    print()


    sl2.mark_today("2025-10-27")
    print(f"Today (on due date): {sl2.today}")
    print(f"Is overdue: {sl2.is_overdue}")
    print()


    sl2.mark_today("2025-10-26")
    print(f"Today (before due date): {sl2.today}")
    print(f"Is overdue: {sl2.is_overdue}")
    print()

    sl2.mark_today("2025-10-28")
    print(f"Today (past due date): {sl2.today}")
    print(f"Is overdue: {sl2.is_overdue}")
    print(sl2)
    print()


    sl3 = SimpleLoan("2099-12-31")
    sl3.mark_today("2050-06-15")
    print(f"Future loan, is_overdue: {sl3.is_overdue}")
    print()


Due date: 2025-10-20
Today not set yet, is_overdue: False

Today: 2025-10-21
Is overdue: True
SimpleLoan(due_date='2025-10-20', today='2025-10-21', is_overdue=True)

Due date: 2025-10-27
Is overdue (before marking today): False

Today (on due date): 2025-10-27
Is overdue: False

Today (before due date): 2025-10-26
Is overdue: False

Today (past due date): 2025-10-28
Is overdue: True
SimpleLoan(due_date='2025-10-27', today='2025-10-28', is_overdue=True)

Future loan, is_overdue: False



### Exercise 12 — Immutable `LibraryCard` number
Create `LibraryCard` with private `__number` (string). Provide only a read-only `number` property. Add optional `pin` with setter that requires 4 digits. Do **not** allow number changes after construction.

In [18]:
class LibraryCard:
    def __init__(self, number: str, pin: str | None = None):

        self.__number = number
        self.__pin = None


        if pin is not None:
            self.pin = pin

    @property
    def number(self) -> str:
        """Get the card number (read-only, immutable)."""
        return self.__number

    @property
    def pin(self) -> str | None:
        """Get the card's PIN (or None if not set)."""
        return self.__pin

    @pin.setter
    def pin(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"PIN must be a string, got {type(value).__name__}")

        if len(value) != 4:
            raise ValueError(
                f"PIN must be exactly 4 characters long, got length {len(value)}"
            )

        if not value.isdigit():
            raise ValueError(f"PIN must contain only digits, got {value!r}")

        self.__pin = value

    def __repr__(self) -> str:
        """Developer-facing representation."""
        pin_str = self.__pin if self.__pin is not None else "None"
        return f"LibraryCard(number={self.__number!r}, pin={pin_str!r})"


if __name__ == "__main__":

    lc = LibraryCard("CARD-0001")
    print(lc.number, lc.pin)
    print(lc)

    print()


    lc.pin = "1234"
    print(lc.number, lc.pin)
    print(lc)

    print()


    lc.pin = "5678"
    print(lc.pin)

    print()


    lc2 = LibraryCard("CARD-0002", "9876")
    print(lc2.number, lc2.pin)
    print(lc2)

    print()


    print()

    lc3 = LibraryCard("CARD-0003", "0000")
    print(f"PIN with zeros: {lc3.pin}")

    lc4 = LibraryCard("CARD-0004", "9999")
    print(f"PIN with nines: {lc4.pin}")

    print()

CARD-0001 None
LibraryCard(number='CARD-0001', pin='None')

CARD-0001 1234
LibraryCard(number='CARD-0001', pin='1234')

5678

CARD-0002 9876
LibraryCard(number='CARD-0002', pin='9876')


PIN with zeros: 0000
PIN with nines: 9999



### Exercise 13 — Title case normalization with `@property`
Add a `title` property to `Book`-like class that always returns title-cased text and stores trimmed private `__title`. Prevent setting titles shorter than 2 characters.

In [19]:
class TitledBook:
    def __init__(self, title: str):

        self.title = title

    @property
    def title(self) -> str:

        return self.__title.title()

    @title.setter
    def title(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"title must be a string, got {type(value).__name__}")

        trimmed = value.strip()

        if len(trimmed) < 2:
            raise ValueError(
                f"title must be at least 2 characters long, "
                f"got {len(trimmed)} character(s): {value!r}"
            )

        self.__title = trimmed

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"TitledBook(title={self.title!r})"


if __name__ == "__main__":
    tb = TitledBook("  the pragmatic programmer ")
    print(tb.title)
    print(tb)

    print()


    tb2 = TitledBook("clean code")
    print(tb2.title)

    print()


    tb3 = TitledBook("DESIGN PATTERNS")
    print(tb3.title)

    print()


    tb4 = TitledBook("   tHe GoD oF sMaLl ThInGs   ")
    print(tb4.title)

    print()


    tb5 = TitledBook("it")
    print(tb5.title)

    print()


    tb.title = "  the pragmatic programmer: your journey to mastery  "
    print(tb.title)
    print()


    tb6 = TitledBook("1984")
    print(tb6.title)

    print()


    tb7 = TitledBook("the lord of the rings: the fellowship of the ring")
    print(tb7.title)
    print()


The Pragmatic Programmer
TitledBook(title='The Pragmatic Programmer')

Clean Code

Design Patterns

The God Of Small Things

It

The Pragmatic Programmer: Your Journey To Mastery

1984

The Lord Of The Rings: The Fellowship Of The Ring



### Exercise 14 — Encapsulated search query history
Create `SearchSession` with *protected* list `_queries`. Provide `add_query(q: str)` that stores trimmed non-empty strings only. Expose a read-only `queries` tuple property.

In [20]:
class SearchSession:
    def __init__(self):
        self._queries = []

    def add_query(self, q: str) -> None:

        if not isinstance(q, str):
            raise TypeError(f"Query must be a string, got {type(q).__name__}")

        trimmed = q.strip()


        if trimmed:
            self._queries.append(trimmed)

    @property
    def queries(self):

        return tuple(self._queries)

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"SearchSession(queries={len(self._queries)})"

if __name__ == "__main__":
    ss = SearchSession()
    print(ss)
    print(ss.queries)

    print()


    ss.add_query("  design patterns ")
    print(ss.queries)

    print()


    ss.add_query("python best practices")
    ss.add_query("  clean architecture  ")
    ss.add_query("refactoring techniques")

    print(ss)
    print(ss.queries)


    print()

    ss.add_query("")
    ss.add_query("   ")
    ss.add_query("\t\n")

    print(f"After adding empty queries: {len(ss.queries)} queries")
    print()


    print("Session queries:")
    for i, query in enumerate(ss.queries, 1):
        print(f"  {i}. {query}")

    print()


    first_query = ss.queries[0]
    print(f"First query: {first_query}")

    print()


    print()

    print(f"Total queries: {len(ss.queries)}")  # 4

    print()


    ss2 = SearchSession()
    ss2.add_query("machine learning")
    ss2.add_query("neural networks")
    ss2.add_query("deep learning")

    print(ss2)
    print(ss2.queries)

    print()

SearchSession(queries=0)
()

('design patterns',)

SearchSession(queries=4)
('design patterns', 'python best practices', 'clean architecture', 'refactoring techniques')

After adding empty queries: 4 queries

Session queries:
  1. design patterns
  2. python best practices
  3. clean architecture
  4. refactoring techniques

First query: design patterns


Total queries: 4

SearchSession(queries=3)
('machine learning', 'neural networks', 'deep learning')



### Exercise 15 — Simple `Tag` with sanitized names
Create `Tag` with private `__name`. Setter should lowercase, strip spaces, and collapse internal whitespace to single spaces. Reject tags longer than 30 characters.

In [21]:
class Tag:
    MAX_LENGTH = 30

    def __init__(self, name: str):
        self.name = name

    @property
    def name(self) -> str:
        """Get the tag name (lowercased and sanitized)."""
        return self.__name

    @name.setter
    def name(self, value: str) -> None:

        if not isinstance(value, str):
            raise ValueError(f"name must be a string, got {type(value).__name__}")


        lowercased = value.lower()


        stripped = lowercased.strip()


        sanitized = " ".join(stripped.split())


        if not sanitized:
            raise ValueError(
                f"name cannot be empty or whitespace-only after sanitization: {value!r}"
            )


        if len(sanitized) > self.MAX_LENGTH:
            raise ValueError(
                f"name must be at most {self.MAX_LENGTH} characters, "
                f"got {len(sanitized)} characters: {sanitized!r}"
            )

        self.__name = sanitized

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Tag(name={self.__name!r})"

if __name__ == "__main__":
    t = Tag("  Object   Oriented   Design   ")
    print(t.name)
    print(t)

    print()


    t2 = Tag("PYTHON PROGRAMMING")
    print(t2.name)

    print()

    t3 = Tag("  web\t\tDevelopment\n\nFrameworks  ")
    print(t3.name)

    print()

    t4 = Tag("machine     learning     models")
    print(t4.name)

    print()

    t5 = Tag("   python   ")
    print(t5.name)

    print()


    max_tag = Tag("a" * 30)
    print(f"Max length tag: {len(max_tag.name)} chars - {max_tag.name}")

    print()


    t6 = Tag("  C++   Design   Patterns  ")
    print(t6.name)

    print()


    t.name = "  FUNCTIONAL   Programming   "
    print(t.name)
    print()


    t7 = Tag("data\t\t\tscience\n\nwith\r\npython")
    print(t7.name)
    print()


object oriented design
Tag(name='object oriented design')

python programming

web development frameworks

machine learning models

python

Max length tag: 30 chars - aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

c++ design patterns

functional programming

data science with python



### Exercise 16 — `BookCopy` status transitions via setter
Create `BookCopy` with private `__status` in {'available','on_loan','missing'}. The `status` setter should only allow transitions:
available → on_loan → available, and *any* → missing. Invalid transitions should be rejected (e.g., raise `ValueError`).

In [22]:
class BookCopy:
    VALID_STATUSES = {'available', 'on_loan', 'missing'}

    ALLOWED_TRANSITIONS = {
        'available': {'on_loan', 'missing'},
        'on_loan': {'available', 'missing'},
        'missing': {'missing'},
    }

    def __init__(self, copy_id: str, status: str = "available"):

        self.copy_id = copy_id

        if status not in self.VALID_STATUSES:
            raise ValueError(
                f"status must be one of {sorted(self.VALID_STATUSES)}, got {status!r}"
            )
        self.__status = status

    @property
    def status(self) -> str:
        """Get the current status of this copy."""
        return self.__status

    @status.setter
    def status(self, value: str) -> None:

        if value not in self.VALID_STATUSES:
            raise ValueError(
                f"status must be one of {sorted(self.VALID_STATUSES)}, got {value!r}"
            )

        current = self.__status
        allowed_next = self.ALLOWED_TRANSITIONS.get(current, set())

        if value not in allowed_next:
            raise ValueError(
                f"cannot transition from '{current}' to '{value}'. "
                f"allowed transitions from '{current}': {sorted(allowed_next)}"
            )

        self.__status = value

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"BookCopy(copy_id={self.copy_id!r}, status={self.__status!r})"

if __name__ == "__main__":
    bc = BookCopy("C-2001")
    print(bc)
    print(f"Status: {bc.status}")

    print()

    bc.status = "on_loan"
    print(bc)

    print()

    bc.status = "available"
    print(bc)

    print()

    bc.status = "missing"
    print(bc)

    print()


    bc2 = BookCopy("C-2002", "available")
    print(bc2)

    print()

    bc2.status = "on_loan"
    print(f"Copy 2 status: {bc2.status}")

    print()


    bc2.status = "missing"
    print(f"Copy 2 status: {bc2.status}")

    print()


    bc3 = BookCopy("C-2003", "missing")
    print(bc3)

    print()


    bc4 = BookCopy("C-2004")
    print(f"Initial: {bc4.status}")

    bc4.status = "on_loan"
    print(f"After checkout: {bc4.status}")

    bc4.status = "available"
    print(f"After return: {bc4.status}")

    bc4.status = "on_loan"
    print(f"After re-checkout: {bc4.status}")


BookCopy(copy_id='C-2001', status='available')
Status: available

BookCopy(copy_id='C-2001', status='on_loan')

BookCopy(copy_id='C-2001', status='available')

BookCopy(copy_id='C-2001', status='missing')

BookCopy(copy_id='C-2002', status='available')

Copy 2 status: on_loan

Copy 2 status: missing

BookCopy(copy_id='C-2003', status='missing')

Initial: available
After checkout: on_loan
After return: available
After re-checkout: on_loan


### Exercise 17 — `Account` balance with non-negative invariant
Create `Account` with private `__balance` (float). Provide `deposit(amount)` and `withdraw(amount)` that update the balance via the `balance` property setter enforcing non-negative invariant and numeric coercion.

In [26]:
class Account:
    def __init__(self, opening_balance=0.0):

        self.balance = opening_balance

    @property
    def balance(self) -> float:
        """Get the current account balance."""
        return self.__balance

    @balance.setter
    def balance(self, value) -> None:

        try:
            balance_float = float(value)
        except (ValueError, TypeError):
            raise ValueError(
                f"balance must be coercible to float, "
                f"got {type(value).__name__}: {value!r}"
            )

        if balance_float < 0.0:
            raise ValueError(f"balance cannot be negative, got {balance_float}")

        self.__balance = balance_float

    def deposit(self, amount) -> None:

        try:
            amount_float = float(amount)
        except (ValueError, TypeError):
            raise ValueError(
                f"deposit amount must be coercible to float, "
                f"got {type(amount).__name__}: {amount!r}"
            )

        if amount_float < 0.0:
            raise ValueError(f"deposit amount cannot be negative, got {amount_float}")


        self.balance = self.__balance + amount_float

    def withdraw(self, amount) -> None:

        try:
            amount_float = float(amount)
        except (ValueError, TypeError):
            raise ValueError(
                f"withdraw amount must be coercible to float, "
                f"got {type(amount).__name__}: {amount!r}"
            )

        if amount_float < 0.0:
            raise ValueError(f"withdraw amount cannot be negative, got {amount_float}")

        new_balance = self.__balance - amount_float

        if new_balance < 0.0:
            raise ValueError(
                f"insufficient funds: cannot withdraw {amount_float} "
                f"from balance {self.__balance}"
            )


        self.balance = new_balance

    def __repr__(self) -> str:
        """Developer-facing representation."""
        return f"Account(balance={self.__balance})"


if __name__ == "__main__":

    a = Account(5)
    print(f"Opening balance: {a.balance}")
    print(a)

    print()


    a.deposit("4.5")
    print(f"After deposit: {a.balance}")
    print(a)

    print()


    a.deposit(2)
    print(f"After deposit: {a.balance}")

    print()


    a.withdraw(3.5)
    print(f"After withdrawal: {a.balance}")

    print()


    a.withdraw("2.0")
    print(f"After withdrawal: {a.balance}")

    print()


    a2 = Account("100.50")
    print(f"Account 2: {a2.balance}")

    a2.deposit(50.25)
    print(f"After deposit: {a2.balance}")

    a2.withdraw(75)
    print(f"After withdrawal: {a2.balance}")

    print()


    a3 = Account()
    print(f"Empty account: {a3.balance}")

    a3.deposit(25.99)
    print(f"After deposit: {a3.balance}")

    print()


    a4 = Account(10.0)
    a4.withdraw(10.0)
    print(f"After full withdrawal: {a4.balance}")

    print()

    a.balance = 50.75
    print(f"After direct assignment: {a.balance}")

    print()


Opening balance: 5.0
Account(balance=5.0)

After deposit: 9.5
Account(balance=9.5)

After deposit: 11.5

After withdrawal: 8.0

After withdrawal: 6.0

Account 2: 100.5
After deposit: 150.75
After withdrawal: 75.75

Empty account: 0.0
After deposit: 25.99

After full withdrawal: 0.0

After direct assignment: 50.75



### Exercise 18 — `HoldRequest` priority (1–5)
Create `HoldRequest` with private `__priority` (int). Property setter must coerce to int and clamp/validate to 1–5. Add read-only `created_by_member_id`.

In [28]:
class HoldRequest:
    def __init__(self, created_by_member_id: int, priority: int = 3):

        self.__created_by_member_id = created_by_member_id


        self.priority = priority

    @property
    def created_by_member_id(self) -> int:

        return self.__created_by_member_id

    @property
    def priority(self) -> int:
        return self.__priority

    @priority.setter
    def priority(self, value) -> None:

        try:
            value = int(value)
        except (ValueError, TypeError):
            raise ValueError("Priority must be an integer between 1 and 5.")


        if value < 1:
            value = 1
        elif value > 5:
            value = 5

        self.__priority = value



hr = HoldRequest(42, 5)
print(hr.created_by_member_id, hr.priority)


hr.priority = "7"
print(hr.priority)

hr.priority = 0
print(hr.priority)


42 5
5
1


### Exercise 19 — `CatalogRecord` with encapsulated fields & snapshot
Create `CatalogRecord` with private fields `__title`, `__author`, `__year` and respective properties (validate year 1450–2100). Add method `snapshot()` that returns a **new dict** with current public state (not the private attributes).

In [33]:
class CatalogRecord:
    def __init__(self, title: str, author: str, year):
        self.title = title
        self.author = author
        self.year = year

    @property
    def title(self) -> str:
        return self.__title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Title must be a non-empty string.")
        self.__title = value.strip()

    @property
    def author(self) -> str:
        return self.__author

    @author.setter
    def author(self, value: str) -> None:
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Author must be a non-empty string.")
        self.__author = value.strip()

    @property
    def year(self) -> int:
        return self.__year

    @year.setter
    def year(self, value) -> None:
        try:
            value = int(value)
        except (TypeError, ValueError):
            raise ValueError("Year must be an integer between 1450 and 2100.")

        if not (1450 <= value <= 2100):
            raise ValueError("Year must be in the range 1450–2100.")

        self.__year = value

        rec = CatalogRecord("Design Patterns", "Gamma et al.", 1994)
        print(rec.snapshot())

### Exercise 20 — `Settings` with encapsulated feature flags
Create `Settings` with private `__flags` dict (keys: 'allow_guest_checkout', 'enable_notifications'). Provide boolean properties for each flag that safely read/write the underlying dict while ensuring boolean values.

In [35]:
class Settings:
    def __init__(self, allow_guest_checkout: bool = False, enable_notifications: bool = True):
        self.__flags = {
            "allow_guest_checkout": bool(allow_guest_checkout),
            "enable_notifications": bool(enable_notifications)
        }

    @property
    def allow_guest_checkout(self) -> bool:
        return self.__flags["allow_guest_checkout"]

    @allow_guest_checkout.setter
    def allow_guest_checkout(self, value) -> None:
        self.__flags["allow_guest_checkout"] = bool(value)

    @property
    def enable_notifications(self) -> bool:
        return self.__flags["enable_notifications"]

    @enable_notifications.setter
    def enable_notifications(self, value) -> None:
        self.__flags["enable_notifications"] = bool(value)


s = Settings()
s.allow_guest_checkout = 1
print(s.allow_guest_checkout, s.enable_notifications)


True True
